In [17]:
from selenium import webdriver
import pyodbc
import pandas as pd
import os
import time

In [ ]:
# Configuração da string de conexão
server = 'SQLGDNCLI'        # Substitua pelo nome do servidor
database = 'WORKSPACE'  # Substitua pelo nome do banco de dados

# Montar a string de conexão com autenticação do Windows
conn_sqlgdncli = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"  # Usar autenticação do Windows
)

# Estabelecer a conexão
sqlgdncli = pyodbc.connect(conn_sqlgdncli)

# Criar um cursor para executar consultas
cursor = sqlgdncli.cursor()

# Exemplo de execução de uma consulta
query = """
    SELECT CAST(NUM_BEN_INS AS VARCHAR(10)) NUM_BEN_INS, FORMAT(DATA_NASCIMENTO, 'ddMMyyyy') DATA_NASCIMENTO, NOME, CPF
    FROM [MERCANTIL\B042304].TE_BUSCA_DCB_NO_SITE
    WHERE ERRO IS NULL
    ORDER BY NUM_BEN_INS
    """

df_dados = pd.read_sql(query, sqlgdncli)  # Ler os resultados diretamente para um DataFrame

# Criação de um data frame vazio
df_final = pd.DataFrame(columns=['NUM_BEN_INS', 'NOME', 'IDC', 'ERRO'])

In [ ]:
# Inicializar o driver do Chrome
driver = webdriver.Chrome()

# Abrir a página web
url = "https://sabiweb.inss.gov.br/sabiweb/relatorio/inicio.view"  # Substitua pela URL da página

for i in range(len(df_dados)):

    driver.get(url)

    # Encontrar o elemento input pelo seu ID (ou outro seletor)
    input_element_nuBeneficio = driver.find_element("name", "nuBeneficio")  # Substitua pelo ID correto
    input_element_dtNascimento = driver.find_element("id", "dtNascimento")  # Substitua pelo ID correto
    input_element_nomeRequerente = driver.find_element("id", "nomeRequerente")  # Substitua pelo ID correto
    input_element_cpf = driver.find_element("id", "cpf")  # Substitua pelo ID correto
    
    linha_selecionada = df_dados.iloc[i]

    # Inserir dados no campo de input
    inserir_beneficio = linha_selecionada['NUM_BEN_INS']
    inserir_dt_nascimento = linha_selecionada['DATA_NASCIMENTO']
    inserir_nome_requerente = linha_selecionada['NOME']
    inserir_cpf = linha_selecionada['CPF']

    input_element_nuBeneficio.send_keys(inserir_beneficio)
    input_element_dtNascimento.send_keys(inserir_dt_nascimento)
    input_element_nomeRequerente.send_keys(inserir_nome_requerente)
    input_element_cpf.send_keys(inserir_cpf)

    repetir_erro = 1

    # Sempre vai repetir quando o captcha for respondido com erro
    while (repetir_erro == 1):

        capctha = input("Digite algo: ")

        input_element_captcha_campo_resposta = driver.find_element("id", "captcha_campo_resposta")  # Substitua pelo ID correto
        inserir_captcha = capctha
        input_element_captcha_campo_resposta.send_keys(inserir_captcha)

        submit_button = driver.find_element("id", "botoes")  
        submit_button.click()

        elemento_msg_erro = driver.find_element("css selector", "span.msgErro")
        elemento_msg_erro = elemento_msg_erro.text

        if "não corresponde ao desafio gerado." not in elemento_msg_erro:
            repetir_erro = 0

    # Criar um data frame de apenas 1 linha para o armazenamento do resultado da consulta
    nova_linha = pd.DataFrame([[linha_selecionada['NUM_BEN_INS'], linha_selecionada['NOME'], i, elemento_msg_erro]],
                              columns=['NUM_BEN_INS', 'NOME', 'IDC', 'ERRO'])
    
    # Insersão do resultado da consulta nas bases internas no SQL Server
    insere = sqlgdncli.cursor()
    insere.fast_executemany=True
    insere.executemany('''INSERT INTO [MERCANTIL\B042304].TE_RESULTADOS_BUSCA_DCB_NO_SITE 
                       (NUM_BEN_INS, NOME, IDC, ERRO) 
                       VALUES (?, ?, ?, ?)''', nova_linha.values.tolist())
    insere.commit()

    # Adição da linha temporária no df final
    df_final = pd.concat([df_final, pd.DataFrame(nova_linha)], ignore_index=True)

    print(elemento_msg_erro)

    time.sleep(5)
    


In [ ]:

df_final

In [21]:
# Fechar o cursor e a conexão
cursor.close()
sqlgdncli.close()

# Fechar o navegador
driver.quit()